How to query the rapidXMM database using python
==============================================

The RapidXMM database can be queried programatically using [HTTP GET requests](http://nxsa.esac.esa.int/nxsa-sl/servlet/get-uls). In this repository you can find a [python module](./api/rapidxmm.py) that uses the requests package for querying the database, consistent with the [Astropy](https://www.astropy.org/) ecosystem.

You can obtain all the information contained in the upper limits database at a given position using lists of R.A. and Dec values:

In [1]:
from api import rapidxmm as rpx

ra = [20.0]  # in degrees
dec = [-4.0] # in degrees

uls = rpx.query_radec(ra, dec)

id,npixel,nside,ra,dec,obsid,obstype,start_date,end_date,eef,area_ratio,band6_exposure,band6_src_counts,band6_bck_counts,band6_ul_sigma1,band6_ul_sigma2,band6_ul_sigma3,band7_exposure,band7_src_counts,band7_bck_counts,band7_ul_sigma1,band7_ul_sigma2,band7_ul_sigma3,band8_exposure,band8_src_counts,band8_bck_counts,band8_ul_sigma1,band8_ul_sigma2,band8_ul_sigma3,start_timestamp,instrum,filt,band6_flags,band7_flags,band8_flags
int64,int64,int64,float64,float64,str10,str4,str19,str19,float64,float64,float64,int64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,int64,str2,str6,int64,int64,int64
15965428508,4625393631,15,20.0006103516,-4.0003853849,9184000003,slew,2009-12-26T05:56:35,2009-12-26T06:39:11,0.85,0.032973174,3.565029,0,0.0,0.60744977,1.24774,2.1685438,2.364278,0,0.0,0.91595656,1.8814322,3.2698865,2.7630625,0,0.0,0.7837593,1.6098908,2.7979536,1261806995,PN,Medium,0,0,0


Alternatively, you can also use SkyCoord's Astropy objects for querying:

In [3]:
from astropy.coordinates import SkyCoord

coord = SkyCoord(ra=ra, dec=dec, unit="deg")
uls = rpx.query_coords(coord)

id,npixel,nside,ra,dec,obsid,obstype,start_date,end_date,eef,area_ratio,band6_exposure,band6_src_counts,band6_bck_counts,band6_ul_sigma1,band6_ul_sigma2,band6_ul_sigma3,band7_exposure,band7_src_counts,band7_bck_counts,band7_ul_sigma1,band7_ul_sigma2,band7_ul_sigma3,band8_exposure,band8_src_counts,band8_bck_counts,band8_ul_sigma1,band8_ul_sigma2,band8_ul_sigma3,start_timestamp,instrum,filt,band6_flags,band7_flags,band8_flags
int64,int64,int64,float64,float64,str10,str4,str19,str19,float64,float64,float64,int64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,int64,str2,str6,int64,int64,int64
15965428508,4625393631,15,20.0006103516,-4.0003853849,9184000003,slew,2009-12-26T05:56:35,2009-12-26T06:39:11,0.85,0.032973174,3.565029,0,0.0,0.60744977,1.24774,2.1685438,2.364278,0,0.0,0.91595656,1.8814322,3.2698865,2.7630625,0,0.0,0.7837593,1.6098908,2.7979536,1261806995,PN,Medium,0,0,0


You can also query using HEALPix cell numbers, following the [nested numbering scheme](https://healpix.jpl.nasa.gov/html/intronode4.htm). RapidXMM uses a HEALPix order of 16 for pointed observations and 15 for slew data.

In [5]:
npixel = [4625393631]
uls = rpx.query_npixels(npixel, obstype="slew")  # By default query_npixels assumes obstype="pointed"

id,npixel,nside,ra,dec,obsid,obstype,start_date,end_date,eef,area_ratio,band6_exposure,band6_src_counts,band6_bck_counts,band6_ul_sigma1,band6_ul_sigma2,band6_ul_sigma3,band7_exposure,band7_src_counts,band7_bck_counts,band7_ul_sigma1,band7_ul_sigma2,band7_ul_sigma3,band8_exposure,band8_src_counts,band8_bck_counts,band8_ul_sigma1,band8_ul_sigma2,band8_ul_sigma3,start_timestamp,instrum,filt,band6_flags,band7_flags,band8_flags
int64,int64,int64,float64,float64,str10,str4,str19,str19,float64,float64,float64,int64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,int64,str2,str6,int64,int64,int64
15965428508,4625393631,15,20.0006103516,-4.0003853849,9184000003,slew,2009-12-26T05:56:35,2009-12-26T06:39:11,0.85,0.032973174,3.565029,0,0.0,0.60744977,1.24774,2.1685438,2.364278,0,0.0,0.91595656,1.8814322,3.2698865,2.7630625,0,0.0,0.7837593,1.6098908,2.7979536,1261806995,PN,Medium,0,0,0


All query functions return Astropy Tables, with a row for data entry in the database at each input position. Since a certain position can be observed multiple times by XMM-Newton, is possible to obtain multiple upper limits for a single position.

Query of 100 random positions in the sky:

In [11]:
import numpy as np

rng = np.random.default_rng()

n = 100
ra = 360 * rng.random(n)
dec = 180 * rng.random(n) - 90
coords = SkyCoord(ra, dec, unit="deg")

uls = rpx.query_coords(coords)

You can query only for a certain XMM-Newton's observation type:

In [14]:
uls = rpx.query_coords(coords, obstype="pointed")

In [16]:
uls = rpx.query_coords(coords, obstype="slew")

And also for a certain detector:

In [20]:
uls = rpx.query_coords(coords, instrum="PN")

In [22]:
uls = rpx.query_coords(coords, instrum="M1")

In [24]:
uls = rpx.query_coords(coords, instrum="M2")

Due to current limitations in the public RapidXMM API, is not possible to query more than ~200 positions at once. Large queries can be done by splitting the list of positions and doing multiple queries:

In [32]:
from astropy.table import vstack

n = 1300
ra = 360 * rng.random(n)
dec = 180 * rng.random(n) - 90
coords = SkyCoord(ra, dec, unit="deg")

start, stop = 0, 200
coords_segment = coords[start:stop]

uls_tables = []
while len(coords_segment):
    uls_tables.append(rpx.query_coords(coords_segment))

    start = stop
    stop = stop + 200
    coords_segment = coords[start:stop]

uls = vstack(uls_tables)